# 安装Opencv

从配置文件创建虚拟环境并且激活

```bash
conda create -f envs.yaml
conda activate py_opencv
```

在 `notebook` 中选择 `python` 解释器，点一下顶上右边的选择解释器，也就是图中的 `py_opencv(Python 3.13.5)` 部分


![notebook head](images\notebookhead.png)


在弹出的菜单中选择 `Python环境` 

![notebook head](images\py_select_1.png)


在弹出的菜单中选择 `py_opencv` 环境

![notebook head](images\py_select_2.png)


# 计算机眼中的图片

## 图像的本质——像素、坐标与颜色

### 像素 (Pixel) - 图像的基本单元

“Pixel”是“Picture Element”（图像元素）的缩写。它是构成数字图像的最小、最基本的单位。你可以把它想象成一个无法再被分割的纯色小方块。我们所看到的任何一张高清、细节丰富的数码照片，放大、放大、再放大，最终看到的都会是这些纯色的小方块。

![pixels show](./images/weixin_screenshot.png)

这个图片里面的坐标就是当前鼠标对应的像素的坐标

**一张图像的核心属性，就是由所有像素的属性共同决定的。**

每个像素，都有两个核心信息：

1. 它的位置：它在这张马赛克拼图的哪个位置？
2. 它的值：它应该显示什么颜色？

其实还有一种图是`矢量图`，这种图没有像素的概念，类似于使用数学公式描述画面。所以其不受图像放大的影响，清晰度不会变。但是在导入opencv的进行处理的时候，都会转为一般的像素图片，所以我们这里不再多说。

### 坐标系 (Coordinate System) - 像素的位置

为了管理千千万万个像素的位置，计算机使用了一个二维坐标系。请注意，这个坐标系和我们初中数学学的直角坐标系有一个关键区别：

1. 原点 (0, 0) 在图像的左上角。
2. X 轴 水平向右延伸，代表图像的宽度 (Width)。
3. Y 轴 垂直向下延伸，代表图像的高度 (Height)。

举个例子，对于一张分辨率为 800 x 600 (宽x高) 的图片：

1. 左上角第一个像素的坐标是 (x=0, y=0)。
2. 右上角最后一个像素的坐标是 (x=799, y=0)。
3. 右下角最后一个像素的坐标是 (x=799, y=599)。
4. 图像正中心的像素坐标大约是 (x=400, y=300)。

但是，存储的时候又会反过来，所以在使用的时候，注意一下就可以了。

### 颜色与通道 (Color & Channels) - 定义像素的值

我们已经知道了像素的位置，那如何定义它的颜色呢？这就是“通道”概念的用武之地。

#### 最简单的情况：灰度图 (Grayscale Image)

一张黑白照片就是一张灰度图。它只有一个颜色通道。

+ 每个像素的值是一个单独的数字，用来表示其亮度。
+ 这个数字的范围通常是 0 到 255 (这被称为8位图像，因为 2^8 = 256)。
+ 0 代表纯黑色，255 代表纯白色，中间的数值则代表不同程度的灰色。

![pixels show](./images/grey_sample.png)

#### 彩色图像 (Color Image)

我们日常见到的大多数图片都是彩色的。计算机通过混合不同强度的三原色光来创造出成千上万种颜色。因此，一张彩色图通常有**三个颜色通道**。

+ 最常见的颜色模型是 RGB (Red, Green, Blue)。
+ ⚠️ 重要陷阱： OpenCV 默认使用的颜色通道顺序是 BGR (Blue, Green, Red)！ 这和很多其他软件（如Photoshop, Matplotlib）的RGB顺序不同。在进行颜色相关的操作时，一定要牢记这一点，否则你会得到奇怪的颜色。
+ 在BGR模型中，每个像素的值不再是一个数字，而是由 三个数字组成的元组 (Tuple) 或列表，分别代表该像素中蓝色、绿色和红色的强度。
+ 每个值的范围同样是 0 到 255。

#### 带有透明度的彩色图 (Four-Channel Image)

一张四通道图，通常被称为 **BGRA** 或 **RGBA** 图。

+ 前三个字母我们已经很熟悉了：B (Blue), G (Green), R (Red)。它们决定了像素的颜色。
+ 新增的第四个字母 A 代表 Alpha 通道。

**Alpha通道不控制颜色，而是控制像素的“不透明度 (Opacity)”或“透明度 (Transparency)”。**

和颜色通道一样，Alpha通道的值通常也是一个 0 到 255 的数字：

+ Alpha = 255 (完全不透明)：这是最“实心”的状态。像素会完全显示它自己的颜色，遮挡住它后面的一切。一张普通的JPG照片里所有像素的Alpha值都可以看作是255。
+ Alpha = 0 (完全透明)：像素是“隐形”的。它不显示任何颜色，像一块完全干净的玻璃，会直接显示出它后面的背景。
+ 0 < Alpha < 255 (半透明)：像素处于半透明状态，像一块有颜色的玻璃或一个水印。它会显示自己的颜色，但同时也会透出背景的颜色。Alpha的值越低，像素就越透明。

#### RGB-D图像-携带深度信息

**RGB-D** 代表 **Red, Green, Blue + Depth**。它不是一张单一的图像，而是一组同步并对齐的数据，通常包含两个部分：

+ 一张RGB（或BGR）图像： 这就是我们已经熟悉的标准彩色图像。它告诉我们场景中每个像素的颜色是什么。
+ 一张深度图 (Depth Map)： 这是新增的关键信息。它是一张单通道的“图像”，但它的像素值不是颜色或亮度，而是距离。

![](./images/RGBD-sample.jpg)

### 总结

**图像就是一个巨大的数字矩阵**

我们把所有知识点串联起来：

一张位图是一个由像素构成的网格（由坐标系定义），网格上每个点都有一个或多个数值来代表其颜色（由通道定义）。

这种 **“网格结构 + 数值”** 的组合，在数学和编程中有一个完美对应的概念——**矩阵 (Matrix)**，或者叫 **多维数组 (Multi-dimensional Array)**。

+ 一张灰度图，可以看作一个 二维矩阵。矩阵的形状是 (height, width)。矩阵中的每个元素就是该像素位置的灰度值（0-255）。
+ 一张彩色图，可以看作一个 三维矩阵。矩阵的形状是 (height, width, 3)。这里的 "3" 就代表B, G, R三个通道。你可以把它想象成3个二维矩阵叠在一起。

